In [14]:
import tensorflow as tf

x = tf.constant(10, dtype=tf.float32)
y = tf.constant(20, dtype=tf.float32)
z = tf.Variable(tf.add(x, y))

# No need for global_variables_initializer() in TensorFlow 2.x

# TensorFlow 2.x automatically initializes variables when they are used in sessions or functions

# Instead of session.run(z), you can simply access the value of z using z.numpy()
print(z.numpy())


30.0


In [15]:
#custom activation functions
def my_softplus(z): 
    return tf.math.log(tf.exp(z) + 1.0)
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))
def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [17]:
#custom layers 
from tensorflow import keras
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [22]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    def build(self, batch_input_shape): 
        self.kernel = self.add_weight(name="kernel", shape=[batch_input_shape[-1], self.units],intializer="glorot_normal")
        self.bias = self.add_weight(name="bias", shape=[self.units], initializer="zeros") 
        super().build(batch_input_shape) # must be at the end
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
        "activation": keras.activations.serialize(self.activation)}

In [26]:
#custom model with subclassclassing api 
class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)
    def call(self, inputs):
        Z = self.hidden1(inputs) 
        for _ in range(1 + 3):
            Z = self.block1(Z) 
            Z = self.block2(Z) 
            return self.out(Z)